# Create and Test Scoring Pipeline and Deploy R Shiny Dashboard App

### Introduction

Now that we have built the machine learning model, stored and deployed it, we can use the model to score new data. 

In this notebook we will:

* Programmatically get the ID's for the deployment space and model deployment that were created in the **`1_Model_Training`** notebook.
* Promote assets required for scoring new data into the deployment space.
* Create a deployable function which will take raw data for scoring, prep it into the format required for the model and score it.
* Deploy the function..
* Create the required payload, invoke the deployed function and return predictions. <br>

In the second part we will:

* Store Shiny assets into the same deployment space.
* Deploy Shiny assets as an app and view the dashboard.

**Sample Materials, provided under license. <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2019, 2020. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp. <br>**

In [1]:
from statsmodels.tsa.arima_model import ARIMAResults

import os
import pandas as pd
import datetime
from ibm_watson_machine_learning import APIClient

token = os.environ['USER_ACCESS_TOKEN']

wml_credentials = {
   "token": token,
   "instance_id" : "openshift",
   "url": os.environ['RUNTIME_ENV_APSX_URL'],
   "version": "3.5"
}

client = APIClient(wml_credentials)

### Set up Deployment Space, Deployments and Assets

The following code programmatically gets the deployment space and the model deployment details which were created in **1_Model_Training**. 
We use the space name and default tags that were used when creating the deployments as specified below. 
If multiple spaces with the same name exist, the code will take the space that was created most recently. Similarly, if multiple deployments within the selected space have the same tag, the most recently created deployment is used. 

Alternatively, the user can manually enter the space and deployment guid's.

The code also promotes some assets into the deployment space, specifically, the dataset with raw data for scoring, the python script file which is used for prepping the data and the metadata that was stored when prepping the data. By promoting these assets into the deployment space, they are available and can be accessed by the deployed function. 

In [2]:
space_name = 'effective_farming_model_space'


Get the space we are working in, which is found using the name that were hardcoded in **1_Model_Training**. If there are multiple spaces with the same name, we take the one that was created most recently. 

If the user would like to use a different space manually set the **space_id**.

Set the space as the default space for working.

In [3]:
for space in client.spaces.get_details()['resources']:

    if space['entity']['name'] ==space_name:
        print("Deployment space with ",space_name,"already exists . .")
        space_uid=space['metadata']['id']
        client.set.default_space(space_uid)

Deployment space with  effective_farming_model_space already exists . .


### Promote Assets to Deployment space

Promote the assets into the deployment space. We will use the prep script for getting the raw data into the format required for scoring. We also need the prep metadata that was saved as json during the prep for training, this ensures that the user inputs specified for prepping the data for training are the same used for scoring. We add these assets into the deployment space.  Also store the raw data dataset in the deployment space.

In [4]:
# we have four models corresponding to the 4 climate variables
asset_temp = client.data_assets.create(name='arima_temp_model.json', file_path='/project_data/data_asset/arima_temp_model.json')
asset_humidity = client.data_assets.create(name='arima_humidity_model.json', file_path='/project_data/data_asset/arima_humidity_model.json')
asset_ws = client.data_assets.create(name='arima_ws_model.json', file_path='/project_data/data_asset/arima_ws_model.json')
asset_precip = client.data_assets.create(name='arima_precip_model.json', file_path='/project_data/data_asset/arima_precip_model.json')

Creating data asset...
SUCCESS
Creating data asset...
SUCCESS
Creating data asset...
SUCCESS
Creating data asset...
SUCCESS


In [5]:
# Asset guid
print('Temperature Asset guid', asset_temp['metadata']['guid'])
print('Humidity Asset guid', asset_humidity['metadata']['guid'])
print('Wind Speed Asset guid', asset_ws['metadata']['guid'])
print('Precipitation Asset guid', asset_precip['metadata']['guid'])

Temperature Asset guid 451ca887-bd88-4401-9311-15b89e7095f6
Humidity Asset guid 0e43d32c-3810-48c6-bf69-205cfed4b306
Wind Speed Asset guid 9e7e2e7a-af72-4bb8-ae53-4290534fad8c
Precipitation Asset guid 847c2356-3feb-4944-8fe7-03f36686d2c5


## Create the Deployable Function

Functions can be deployed in Watson Machine Learning in the same way models can be deployed. The python client or REST API can be used to send data to the deployed function. Using the deployed function allows us to prepare the data and pass it to the model for scoring all within the deployed function.

We start off by creating the dictionary of default parameters to be passed to the function. We get the ID's of all assets that have been promoted into the deployment space. We also add the model deployment ID and space ID information into the dictionary.

In [6]:
# Metadata creation
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + token}
assets_dict = {'asset_temp_id' : asset_temp['metadata']['guid'], 'asset_temp_name' : 'arima_temp_model.json',
               'asset_humidity_id' : asset_humidity['metadata']['guid'], 'asset_humidity_name' : 'arima_humidity_model.json', 
               'asset_ws_id' : asset_ws['metadata']['guid'], 'asset_ws_name' : 'arima_ws_model.json',
               'asset_precip_id' : asset_precip['metadata']['guid'], 'asset_precip_name' : 'arima_precip_model.json',
              }
wml_credentials["instance_id"] = "openshift"
ai_parms = {'wml_credentials' : wml_credentials,'header' : header, 'space_id' : space_uid, 'assets' : assets_dict}

### Scoring Pipeline Function

The function below takes 5 hours weather data and outputs next 5 hours weather. It prepares raw data, load model, execute model forecast and generate prediction for weather.

The following rules are required to make a valid deployable function:

* The deployable function must include a nested function named "score".
* The score function accepts a list.
* The list must include an dictionary with the name "values". "values" include a dictionary with the name "test_data".
* Scoring function returns a list with a dictionary key `predictions` and has two key-value pairs: `values` contains forecast weather values and `time` contains time of forecast.


In [7]:
client.data_assets.download(ai_parms['assets']['asset_precip_id'], ai_parms['assets']['asset_precip_name'])

Successfully saved data asset content to file: 'arima_precip_model.json'


'/home/wsuser/work/arima_precip_model.json'

In [8]:
loaded_model = ARIMAResults.load('/home/wsuser/work/arima_precip_model.json')

In [9]:
loaded_model.forecast(steps=5)

2018-06-28 01:00:00    0.001263
2018-06-28 02:00:00    0.001169
2018-06-28 03:00:00    0.000960
2018-06-28 04:00:00    0.000782
2018-06-28 05:00:00    0.000637
Freq: 60T, dtype: float64

In [10]:
def deploy_fn(parms=ai_parms):
    """
        You will deploy this function.
        -------------------------------
        :return score function
    """ 
    try:
         
        import subprocess
        subprocess.check_output( "pip install -I statsmodels==0.11.1 --user", stderr=subprocess.STDOUT, shell=True)
        import pandas as pd
        import numpy as np
        import json
        import requests
        import os

        from statsmodels.tsa.statespace.sarimax import SARIMAX
        from statsmodels.tsa.arima_model import ARIMAResults
        
        from ibm_watson_machine_learning import APIClient
        client = APIClient(parms["wml_credentials"])
        client.set.default_space(parms['space_id'])
    
    except subprocess.CalledProcessError as e:
    
        return { "error" : "subprocess.CalledProcessError:\n\n" + "cmd:\n" + e.cmd + "\n\noutput:\n" + e.output.decode() }
    
    except Exception as e:
    
        return { "error" : repr( e ) }
    
    def score(payload):
        
        
        # Temperature 
        forecast_value = []
        forecast_period = [] 
        # Get all model paths
        temp_model_path = client.data_assets.download(parms['assets']['asset_temp_id'], parms['assets']['asset_temp_name'])
        humidity_model_path = client.data_assets.download(parms['assets']['asset_humidity_id'], parms['assets']['asset_humidity_name'])
        ws_model_path = client.data_assets.download(parms['assets']['asset_ws_id'], parms['assets']['asset_ws_name'])
        precip_model_path = client.data_assets.download(parms['assets']['asset_precip_id'], parms['assets']['asset_precip_name'])
        
        for i in [temp_model_path, humidity_model_path, ws_model_path, precip_model_path]:
            # load the model
            loaded_model = ARIMAResults.load(i)
            # extend model if there is any past history data
            if payload['input_data'][0]['values'][0] != 0:
                input_data = pd.read_json(payload['input_data'][0]['values'][0])
                input_data.index.freq = '60Min'
                loaded_model = loaded_model.extend(input_data)
            # forecast next 5-hour weather
            result = loaded_model.forecast(steps=5)
            for j in range(len(result)):
                forecast_value.append(result[j])
                forecast_period.append(str(result.index[j]))
        # return response
    
        # score_response = {'predictions': [{ 'values': [['a']]}]} 
        return {'predictions': [{ 'values': [forecast_value], 'time': [forecast_period] }]}
    return score

## Deploy the Function

The user can specify the name of the function and deployment in the code below. As we have previously seen, we use tags in the metadata to allow us to programmatically identify the deployed function. 

### Get the ID of software specification to be used with the function

The Software Specification refers to the runtime used in the Notebook, WML training and WML deployment. It contains details about the runtime platform, framework versions, other packages used and any custom library used in the concerned runtime.

Our notebooks use the `default_py3.6` software specification. When we deploy our function we want it to have the same software specification as the notebooks. We get the ID of the notebook software specification and include it in the metadata when storing the function.

In [11]:
software_spec_id = client.software_specifications.get_uid_by_name("default_py3.7")

In [12]:
# Store function details

meta_data = {
    client.repository.FunctionMetaNames.NAME : 'Function for predicting weather',
    client.repository.FunctionMetaNames.TAGS : ['weather_function_tags'],
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: software_spec_id

}

function_details = client.repository.store_function( meta_props=meta_data, function=deploy_fn)
# Get function id
function_id = function_details["metadata"]["id"]

In [13]:
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: 'complete model deployment',
   client.deployments.ConfigurationMetaNames.TAGS : ['complete_forecast_function_deployment_tag'],
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

# deploy the function
function_deployment_details = client.deployments.create(artifact_uid=function_id, meta_props=meta_props)



#######################################################################################

Synchronous deployment creation for uid: '6d670556-0285-4df9-9a0e-edbed9c6811a' started

#######################################################################################


initializing........
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='7b4a1382-8bd6-443a-9058-d6e8b731e2b2'
------------------------------------------------------------------------------------------------




### Score New Data

Get the guid of the deployed function, create the payload and use the python client to score the data. The deployed function returns the weather prediction. 

The payload contains two values. The first is the effective date for scoring. This is the date that the prediction is computed. The scoring observation window and forecast horizon are calculated from this date. 
The second value contains weather predictions. 

In [14]:
scoring_deployment_id = client.deployments.get_uid(function_deployment_details)
client.deployments.get_details(scoring_deployment_id)

{'entity': {'asset': {'id': '6d670556-0285-4df9-9a0e-edbed9c6811a'},
  'custom': {},
  'deployed_asset_type': 'function',
  'hardware_spec': {'id': 'Not_Applicable', 'name': 'XXS', 'num_nodes': 1},
  'name': 'complete model deployment',
  'online': {},
  'space_id': '9f809528-b691-4fe6-ae13-36411e5f0cdc',
  'status': {'online_url': {'url': 'https://internal-nginx-svc:12443/ml/v4/deployments/7b4a1382-8bd6-443a-9058-d6e8b731e2b2/predictions'},
   'state': 'ready'}},
 'metadata': {'created_at': '2021-08-03T13:34:08.383Z',
  'id': '7b4a1382-8bd6-443a-9058-d6e8b731e2b2',
  'modified_at': '2021-08-03T13:34:08.383Z',
  'name': 'complete model deployment',
  'owner': '1000331209',
  'space_id': '9f809528-b691-4fe6-ae13-36411e5f0cdc',
  'tags': ['complete_forecast_function_deployment_tag']}}

In [15]:
payload_scoring={"input_data": [{"values": [0]}]}
print(payload_scoring['input_data'][0]['values'][0])
#payload_metadata = {client.deployments.ScoringMetaNames.INPUT_DATA: payload}
# score
funct_output = client.deployments.score(scoring_deployment_id, payload_scoring)
funct_output

0


{'predictions': [{'values': [[69.60014768188962,
     69.39219749980322,
     69.47374511323164,
     69.8514987930284,
     70.50145885728497,
     100.78789643511031,
     100.76381035226314,
     99.92956415786254,
     98.34218092237177,
     96.10998673259584,
     13.623792231063465,
     13.32651551084038,
     13.129206215966416,
     12.918194955946312,
     12.71351004756711,
     0.0012627225856711448,
     0.0011692884875361988,
     0.0009597709082982593,
     0.0007820719636892898,
     0.0006369730205670305]],
   'time': [['2018-06-28 01:00:00',
     '2018-06-28 02:00:00',
     '2018-06-28 03:00:00',
     '2018-06-28 04:00:00',
     '2018-06-28 05:00:00',
     '2018-06-28 01:00:00',
     '2018-06-28 02:00:00',
     '2018-06-28 03:00:00',
     '2018-06-28 04:00:00',
     '2018-06-28 05:00:00',
     '2018-06-28 01:00:00',
     '2018-06-28 02:00:00',
     '2018-06-28 03:00:00',
     '2018-06-28 04:00:00',
     '2018-06-28 05:00:00',
     '2018-06-28 01:00:00',
     '2018-06

**The R Shiny Dashboard invokes this scoring pipeline for visualizing the results.***

# Deploy Shiny App

In [16]:
r_shiny_deployment_name='Effective_Farming_Shiny_App'

### Store the App

Create the associated metadata and store the dashboard zip file in the deployment space. 

In [17]:
# Meta_props to store assets in space 
meta_props = {
    client.shiny.ConfigurationMetaNames.NAME: "Effective_Farming_Shiny_Assets",
    client.shiny.ConfigurationMetaNames.DESCRIPTION: 'Store shiny assets in deployment space' # optional
}
app_details = client.shiny.store(meta_props, '/project_data/data_asset/effective-farming-monitor-crop-growth-dashboard.zip')

Creating Shiny asset...
SUCCESS


### Deploy the App

Create the metadata for the Shiny deployment by providing  name, description, R-Shiny options and Hardware specifications. R-Shiny configuration provides options on whom you want to share the dashboard with, they are: 
<br>
* Anyone with the link 
* Authenticated users 
* Collaborators in this deployment space

In [18]:
# Deployment metadata.
deployment_meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: r_shiny_deployment_name,
    client.deployments.ConfigurationMetaNames.DESCRIPTION: 'Deploy Effective Farming Dashboard',
    client.deployments.ConfigurationMetaNames.R_SHINY: { 'authentication': 'anyone_with_url' },
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { 'name': 'S', 'num_nodes': 1}
}


# Create the deployment.
app_uid = client.shiny.get_uid(app_details)
rshiny_deployment = client.deployments.create(app_uid, deployment_meta_props)



#######################################################################################

Synchronous deployment creation for uid: '7da91310-6490-40f9-8acc-cd4213547f68' started

#######################################################################################


initializing......
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='54a082b8-5c4e-4aeb-bbf7-088ac625f1e5'
------------------------------------------------------------------------------------------------




### Launch Shiny App
Now that the dashboard is deployed, it can be accessed through the web browser. The app URL can be found by navigating to the deployed app in the deployment space. 

Open the Navigation Menu, under ***Analytics*** select ***Analytics deployments -> effective_farming_model_space -> Deployments -> Effective_Farming_Shiny_Assets*** to find the dashboard URL.

Alternatively, the path for the app URL can be found from the deployment metadata created in the previous cell. This path should be appended to the user's Cloud Pak for Data hostname to get the complete app URL. To get the path, run the cell below:

In [19]:
print("{HOSTNAME}"+"/ml/v4/deployments/"+rshiny_deployment['metadata']['id'] + '/r_shiny')


{HOSTNAME}/ml/v4/deployments/54a082b8-5c4e-4aeb-bbf7-088ac625f1e5/r_shiny
